In [0]:
dbutils.widgets.removeAll()

In [0]:
dbutils.widgets.text("config", "/Workspace/Users/alex.lopes@databricks.com/WorkspaceInventory/conf/")
dbutils.widgets.text("catalog_schema", "shared.bigc")
widgets = dbutils.widgets.getAll()
catalog_schema = widgets['catalog_schema']

In [0]:
from databricks.sdk import WorkspaceClient, AccountClient
from pprint import pprint
import yaml
import traceback
from pyspark.sql.functions import *
import os

In [0]:
print("Config:", widgets["config"])
configs = {}
for file in os.listdir(widgets["config"]):
  print("File:", file)
  with open(widgets['config'] + file, "r") as f:
    configs[file] = yaml.safe_load(f.read())

pprint(configs)

Config: /Workspace/Users/alex.lopes@databricks.com/WorkspaceInventory/conf/
File: alerts.yml
File: clusters.yml
File: experiments.yml
File: recipients.yml
File: workflows.yml
File: storage_credentials.yml
File: warehouses.yml
File: secrets.yml
File: shares.yml
File: repos.yml
File: pipelines.yml
File: notebooks.yml
File: models.yml
File: ip_access_lists.yml
File: instance_pools.yml
File: global_init_scripts.yml
{'alerts.yml': [{'api': {'call': 'w.alerts.list',
                         'write': {'mode': 'overwrite',
                                   'table': 'alerts'}}}],
 'clusters.yml': [{'api': {'call': 'w.clusters.list',
                           'write': {'mode': 'overwrite',
                                     'table': 'clusters'}}}],
 'experiments.yml': [{'api': {'call': 'w.experiments.list_experiments',
                              'write': {'mode': 'overwrite',
                                        'table': 'experiments'}}}],
 'global_init_scripts.yml': [{'api': {'call': 

In [0]:

columns = ["object_type"]
objects = []

for key, values in configs.items():
  key = key.replace(".yaml", "").replace(".yml", "")
  for conf in values:
    table = conf['api']['write']['table']
    table = f"{catalog_schema}.{table}"
    print("Table:", table)
    if spark.catalog.tableExists(table):      
      objects.append(f"select '{key}' as object_type from {table}")

objects = " union all ".join(objects)
print("Objects SQL:", objects)
if objects:
  spark.sql(f"create or replace view {catalog_schema}.objects as " + objects)
  df_objects = spark.read.table(f"{catalog_schema}.objects") 

  df_objects.display()

Table: alerts
Table: clusters
Table: experiments
Table: recipients
Table: workflows
Table: storage_credentials
Table: warehouses
Table: secrets
Table: shares
Table: repos
Table: pipelines
Table: notebooks
Table: models
Table: ip_access_lists
Table: instance_pools
Table: global_init_scripts


object_type
alerts
alerts
alerts
alerts
alerts
alerts
alerts
alerts
alerts
alerts
